## Estimation de la population fr à partir du recensement 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import geopandas as gpd
import copy
import typing 

In [ ]:
def load_data()-> dict:
    """
    Renvoie un dict : clés correspondent aux noms de onglets du fichier
    et les valeurs = aux données des feuilles de calcul
    """
    data = pd.Series([""])